# 気象と家計支出の相関（Correlation）とヒートマップ

### Colaboratory上で実行する場合
この場合，ファイルをGoogle Driveに置くことを勧めます。この方法は次を参照してください。<br>
本書のサポートサイト:https://github.com/ohmsha/MHBooks → Tips → UsageIDE → "Colabの使い方"<br>
このページに，Goodle Driveに置いたファイルの読込み方の説明があります。

## 相関係数と無相関の検定
相関係数$\hat{\rho}$には次の性質がある。
$$
- 1 \le \hat{\rho}  \le 1
$$
統計検定量は次を用いる
$$
t_{uc} = \frac{{ |\hat \rho _{xy} | \sqrt {N - 2} }}{{\sqrt {1 - \hat \rho _{xy} ^2 } }}
$$
これは，自由度$(N-2)$の$t$分布に従う

ピアソンの相関係数と帰無仮説(無相関)と設定した場合のp値　の両方を同時に計算する  
scipy.stats.pearsonr  
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### 親子の身長の相関の検定


In [ ]:
x = np.array([168, 172, 181, 179, 166, 185, 177, 176, 169, 161])
y = np.array([111, 125, 129, 120, 126, 133, 130, 116, 118, 115])
corr, pvalue = stats.pearsonr(x,y)
print('corr. coef.=',corr, '  pvalue=',pvalue)

alp = 0.05 のとき，pvalue < alp より，H0（無相関）を棄却，alp = 0.01のとき棄却できない

In [ ]:
plt.scatter(x,y)
plt.xlabel('Height of Faters')
plt.ylabel('Height of Sons')
#plt.savefig('fig_SD_CorrelationEx01.pdf')

## stats.pearsonrのpvalueの計算を検証する
上記のstats.pearsonr(x,y)の計算過程を見る。このため，下記の統計検討量tucをまず計算する。

In [ ]:
n = x.size
tuc = corr*np.sqrt((n-2)/(1-corr*corr))
tuc

#### tucが自由度$N-2$の$t$分布に従うこと，および，tucの問題設定では両側検定であることに留意
両側検定ゆえ，probを2倍している。

In [ ]:
prob = 1-stats.t.cdf(tuc,n-2)
2*prob 

上記の結果は，pvalueと一致しているとみなしてよく，pvalueの計算の過程がわかる。

## 気象と家計支出の相関行列とヒートマップ
2年分、月毎のデータ

In [ ]:
fname = 'weather_items.csv'
df = pd.read_csv(fname, index_col='date', parse_dates=[0], comment='#', encoding='SHIFT-JIS')
df.head()

In [ ]:
corr_mat = df.corr()
corr_mat

### Seabornのカラーマップ
http://seaborn.pydata.org/tutorial/color_palettes.html<br>
カラーマップ http://seaborn.pydata.org/tutorial/color_palettes.html<br>
ちょっと古いカラーマップ https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html<br>

In [ ]:
fig = plt.subplots(figsize=(10,10))
heatmap = sns.heatmap(corr_mat, annot=True, square=True, cmap='bwr')
#plt.savefig('fig_SD_CorrelationHeatmap.pdf')
#plt.show()

In [ ]:
#fig = plt.subplots(figsize=(6,4))
pg = sns.pairplot(corr_mat, height=1, aspect=1)
#plt.savefig('fig_SD_ScatterMatrix.pdf')
#plt.show()

In [ ]:
df.plot(subplots=True, figsize=(12,12))

クロス集計を行うために，気象データを削除する
次に，groupby機能を用いて，四季毎にデータをまとめる

In [ ]:
df_1 = df.drop(['max_temperature', 'sum_rainfall', 'mean_humidity'], axis=1)
df_season = df_1.groupby('season')

メンバ関数sum()を用いて，クロス集計を行う（あまり意味が認められない）

In [ ]:
df_season_sum = df_season.sum()
df_season_sum

季節ごとの家計支出を見る

In [ ]:
df_season.plot.bar(rot=0, figsize=(12,4))